# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

In [1]:
import pandas as pd
import numpy as np
import xlwings as xw

from pathlib import Path

In [2]:
path = Path('recipes_sample_with_tags_ingredients.csv')
path

WindowsPath('recipes_sample_with_tags_ingredients.csv')

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [3]:
wb1 = xw.Book("себестоимостьА_в1.xlsx")
sheet = wb1.sheets["Рецептура"]
temp = sheet.range("G7:O10").options(np.array).value
temp

array([[0.2  , 0.4  , 0.02 , 0.02 , 0.01 , 0.005, 0.1  ,   nan,   nan],
       [0.07 , 0.25 , 0.01 , 0.06 , 0.015,   nan, 0.06 , 0.05 ,   nan],
       [0.12 , 0.22 , 0.005,   nan, 0.009, 0.005,   nan, 0.05 , 0.03 ],
       [0.12 , 0.23 , 0.01 ,   nan, 0.015,   nan,   nan,   nan, 0.05 ]])

In [4]:
unit_price = sheet.range("G14:O14").options(np.array).value
unit_price

array([  7.,  10., 184.,  19.,   7., 290., 105.,  83., 260.])

In [5]:
cost_price = np.nan_to_num(temp * unit_price).sum(axis=1)
cost_price

array([21.48 , 16.525, 17.423, 18.085])

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [6]:
sheet.range("T7:T10").options(transpose=True).value = cost_price
sheet.range("T6").value = "Себестоимость"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [ ]:
sheet.range("T4:T6").api.merge()
sheet.range("T4:T6").color = (255,200,0)
sheet.range("T7:T10").color = (244,244,180)

4. Выполнить 3 с помощью "протягиваемых" формул.

In [ ]:
sheet.range("U7").formula = "=SUMPRODUCT(G7:O7, $G$14:$O$14)"
sheet.range("U7").api.autofill(destination=sheet.range("U7:U10").api , type=xw.constants.AutoFillType.xlFillDefault)

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [ ]:
reviews_sample = pd.read_csv("reviews_sample (2).csv", sep=",", parse_dates=['date'])
reviews_sample = reviews_sample.rename(columns={"Unnamed: 0": "index"})
reviews_sample = reviews_sample.set_index("index")
reviews_sample

In [ ]:
recipes_sample_with_tags_ingredients = pd.read_csv("recipes_sample_with_tags_ingredients.csv", sep=",", parse_dates=['submitted'])
recipes_sample_with_tags_ingredients = recipes_sample_with_tags_ingredients.rename(columns={"Unnamed: 0": "index"})
recipes_sample_with_tags_ingredients = recipes_sample_with_tags_ingredients.set_index("index")
recipes_sample_with_tags_ingredients

In [ ]:
recipes_sample_with_tags_ingredients.drop(["contributor_id", "n_steps","n_tags","tags","ingredients"], axis=1, inplace=True)
recipes_sample_with_tags_ingredients

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [ ]:
path = "recipes.xlsx"
coeff = 0.05
writer = pd.ExcelWriter(path)
new_recipes = recipes_sample_with_tags_ingredients.sample(frac = coeff)
new_recipes.to_excel(writer, sheet_name='Рецепты') 

In [ ]:
new_reviews = reviews_sample.sample(frac = coeff)
new_reviews.to_excel(writer, sheet_name='Отзывы') 
writer.save()

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
wb = xw.Book(path)
sheet = wb.sheets["Рецепты"]
sheet.range("H1").value = "seconds_assign"
sheet.range("H1").api.font_object.font_style.set('bold')
sheet.range("H2:H1501").options(transpose=True).value = (new_recipes["minutes"] * 60).to_numpy()

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
sheet.range("I1").value = "seconds_formula"
sheet.range("I1").api.font_object.font_style.set('bold')
sheet.range("I2").formula = "=D2*60"
sheet.range("I2").api.autofill(type=xw.constants.AutoFillType.xlFillDefault, destination=sheet.range("I2:I1501").api)

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
sheet.range("J1").value = "n_reviews"
sheet.range("J1").api.font_object.font_style.set('bold')

n = str(len(new_reviews)+1)
sheet.range("J2").formula = f'=COUNTIF(Отзывы!$C$2:$C${n}, "="&B2)'
sheet.range('J2').api.autofill(type=xw.constants.AutoFillType.xlFillDefault, destination=sheet.range("J2:J1501").api)

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
sheet['H1:J1'].api.horizontalalignment = xw.constants.HAlign.xlHAlignCenter

table = xw.Range("A1:I1501")
table.columns[1].column_width = 50
for i in range(4, 9):
    table.columns[i].column_width = 25

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
from enum import Enum

class Colors(Enum):
    GREEN = (0, 255, 0)
    YELLOW = (255, 255, 0) 
    RED = (255, 0, 0)

In [ ]:
values = sheet.range("D2:D1501").value

for i in range(len(values)):
    
    if values[i] <= 5:
        sheet.range(f"D{i+2}").color = Colors.GREEN.value
    
    elif 5 < values[i] <= 10:
        sheet.range(f"D{i+2}").color = Colors.YELLOW.value
    
    else:
        sheet.range(f"D{i+2}").color =  Colors.RED.value

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
from typing import Union
def validate(data : Union[xw.main.Sheet, xw.main.Book]) -> None:
    
    sheet = None
    if isinstance(data, xw.main.Book):
        try:
            sheet = data.sheets["Отзывы"]
        except Exception as e:
            raise ValueError("Нет отзывов в данном файле Excel!")
    
    elif isinstance(data, xw.main.Sheet):
        sheet = data
    
    else:
        raise ValueError("data должен принимать либо Book, либо Sheet!")

    rng = sheet.range('A1')
    n = rng.current_region.end('down').row

    sheet.range("H2").formula = '=COUNTIF(Рецепты!$B$2:Рецепты!$B$1501,"="&C2)>0'
    sheet.range("H2").api.autofill(type=xw.constants.AutoFillType.xlFillDefault, destination=sheet.range(f"H2:H{n}").api)

    for i in range(2, int(n)+1):
        
        rating = sheet.range(f"E{i}").value
        exist_flag = sheet.range(f"H{i}").value
        
        if rating not in range(0,6) or not exist_flag:
            sheet.range(f"A{i}:G{i}").color = Colors.RED.value
    
    sheet.range('H:H').value = " "
    
new_sheet = wb.sheets["Отзывы"]
#validate(new_sheet)
validate(wb)
wb.save(path)
wb.close()

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [ ]:
with open('recipes_model.csv', 'r') as file:
    reader = csv.reader(file)
    data = list(reader)

book = xw.Book()

sheet = book.sheets['Модель книги']
sheet.range('A2').value = data

book.save('recipes_model.xlsx')
book.close()

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 